In [2]:
!wget https://www.kaggle.com/competitions/amex-default-prediction/data?select=train_data.csv

--2022-08-13 08:13:19--  https://www.kaggle.com/competitions/amex-default-prediction/data?select=train_data.csv
Resolving www.kaggle.com (www.kaggle.com)... 35.244.233.98
Connecting to www.kaggle.com (www.kaggle.com)|35.244.233.98|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘data?select=train_data.csv.1’

     0K .......... .......... .......... .......... ........    300K=0.2s

2022-08-13 08:13:20 (300 KB/s) - ‘data?select=train_data.csv.1’ saved [50064]



In [ ]:
%pip install catboost -U

In [204]:
!cat test1/test_data.zip* > ~/test.zip

In [218]:
!unzip 'test_data.csv.zip'

Archive:  test_data.csv.zip
  inflating: test_data.csv           


In [31]:
import numpy as np # linear algebra
import pandas as pd
import joblib
import datetime
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score
import copy
import gc

from scipy.sparse import csr_matrix
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.model_selection import StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import os
import time

# Essential DS libraries
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
import torch
from catboost import CatBoostClassifier
from catboost import Pool, cv
from sklearn.preprocessing import LabelEncoder
import tqdm

from sklearn.linear_model import LogisticRegression
import torch.nn.functional as F
import math

import torch
import pandas as pd
import torch.nn as nn
from torch.autograd import Variable
#from tqdm.notebook import tqdm
from sklearn.metrics import roc_auc_score
import random
import torch
import pandas as pd
import torch.nn as nn
from torch.autograd import Variable
#from tqdm.notebook import tqdm
from sklearn.metrics import roc_auc_score
import random
torch.manual_seed(42)
random.seed(42)
np.random.seed(42)
torch.cuda.manual_seed(42)
torch.cuda.manual_seed_all(42)
rng = np.random.default_rng(seed=42)

In [157]:
#!c1.8
df=[]
for i in range(1,23):
    path=f'Test/{i}.parquet'
    df.append(pd.read_parquet(path))
    
test=pd.concat(df)

In [222]:
#!c1.8
test=pd.read_csv('test_data.csv')

[0;31mKernelOutOfMemory[0m: Kernel ran out of memory and has been restarted. If the restart fails, restart the kernel from the Kernel menu.
If the error persists, try choosing a different configuration or optimizing your code.

In [227]:
#!c1.8
chunksize = 3e6
print('chunksize=', chunksize)
test=[]
def process_big_csv(chunk, dest_file):
    print('===')
    #---convert float64 to float32--------
    float64_cols = chunk.select_dtypes(include=['float64']).columns.tolist()
    chunk[float64_cols] = np.float16(chunk[float64_cols].values)
    #---convert int64 to int32
    int64_cols = chunk.select_dtypes(include=['int64']).columns.tolist()
    chunk[int64_cols] = np.int32(chunk[int64_cols].values)
    
    #-- save to parquet file
#     table = pa.Table.from_pandas(chunk)
#     pq.write_table(table, dest_file, compression = 'GZIP')
    
    test.append(chunk)
    gc.collect()

test_file = 'test_data.csv'

with open(test_file, 'r') as csv_file:
    reader= pd.read_csv(csv_file, chunksize=chunksize) 
    for i, chunk in enumerate(reader):
        process_big_csv(chunk, "")
        
test=pd.concat(test)

chunksize= 3000000.0
===
===
===
===


/kernel/lib/python3.8/site-packages/ml_kernel/kernel.py:873: UserWarning: The following variables cannot be serialized: reader
  warnings.warn(message)


In [ ]:
#!c1.8


In [ ]:
#!c1.8


In [219]:
#!c1.8
dtype.keys()

dict_keys(['P_2', 'D_39', 'B_1', 'B_2', 'R_1', 'S_3', 'D_41', 'B_3', 'D_42', 'D_43', 'D_44', 'B_4', 'D_45', 'B_5', 'R_2', 'D_46', 'D_47', 'D_48', 'D_49', 'B_6', 'B_7', 'B_8', 'D_50', 'D_51', 'B_9', 'R_3', 'D_52', 'P_3', 'B_10', 'D_53', 'S_5', 'B_11', 'S_6', 'D_54', 'R_4', 'S_7', 'B_12', 'S_8', 'D_55', 'D_56', 'B_13', 'R_5', 'D_58', 'S_9', 'B_14', 'D_59', 'D_60', 'D_61', 'B_15', 'S_11', 'D_62', 'D_65', 'B_16', 'B_17', 'B_18', 'B_19', 'B_20', 'S_12', 'R_6', 'S_13', 'B_21', 'D_69', 'B_22', 'D_70', 'D_71', 'D_72', 'S_15', 'B_23', 'D_73', 'P_4', 'D_74', 'D_75', 'D_76', 'B_24', 'R_7', 'D_77', 'B_25', 'B_26', 'D_78', 'D_79', 'R_8', 'R_9', 'S_16', 'D_80', 'R_10', 'R_11', 'B_27', 'D_81', 'D_82', 'S_17', 'R_12', 'B_28', 'R_13', 'D_83', 'R_14', 'R_15', 'D_84', 'R_16', 'B_29', 'S_18', 'D_86', 'D_87', 'R_17', 'R_18', 'D_88', 'B_31', 'S_19', 'R_19', 'B_32', 'S_20', 'R_20', 'R_21', 'B_33', 'D_89', 'R_22', 'R_23', 'D_91', 'D_92', 'D_93', 'D_94', 'R_24', 'R_25', 'D_96', 'S_22', 'S_23', 'S_24', 'S_25', 

In [230]:
#!c1.8
for i in dtype.keys():
    if i=='target' :continue
    if dtype[i]=='float16':
        test[i]=np.float16(test[i])

In [231]:
#!c1.8
joblib.dump(test,'small_test.sav')

['small_test.sav']

In [160]:
#!c1.8
len(test.customer_ID.unique())

2

In [22]:
!unzip 'train_labels.csv.zip'

Archive:  train_labels.csv.zip
  inflating: train_labels.csv        


In [14]:
#!c1.8
df=read_csv("train_data.csv")

NameError: name 'read_csv' is not defined

In [16]:
#!c1.8
import pandas as pd

In [18]:
#!c1.8
df=pd.read_csv("train_data.csv")

In [25]:
#!c1.8
df.head()

,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,...,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145,target
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-03-09,0.938469,0.001733,0.008724,1.006838,0.009228,0.124035,0.008771,0.004709,...,NaN,NaN,0.002427,0.003706,0.003818,NaN,0.000569,0.000610,0.002674,0
1,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-04-07,0.936665,0.005775,0.004923,1.000653,0.006151,0.126750,0.000798,0.002714,...,NaN,NaN,0.003954,0.003167,0.005032,NaN,0.009576,0.005492,0.009217,0
2,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-05-28,0.954180,0.091505,0.021655,1.009672,0.006815,0.123977,0.007598,0.009423,...,NaN,NaN,0.003269,0.007329,0.000427,NaN,0.003429,0.006986,0.002603,0
3,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-06-13,0.960384,0.002455,0.013683,1.002700,0.001373,0.117169,0.000685,0.005531,...,NaN,NaN,0.006117,0.004516,0.003200,NaN,0.008419,0.006527,0.009600,0
4,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-07-16,0.947248,0.002483,0.015193,1.000727,0.007605,0.117325,0.004653,0.009312,...,NaN,NaN,0.003671,0.004946,0.008889,NaN,0.001670,0.008126,0.009827,0


In [21]:
#!c1.8
len(df)

5531451

In [23]:
#!c1.8
labels=pd.read_csv('train_labels.csv')

In [24]:
#!c1.8
df=df.merge(labels, how='left',left_on='customer_ID',right_on='customer_ID')

In [26]:
#!c1.8

df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 5531451 entries, 0 to 5531450
Columns: 191 entries, customer_ID to target
dtypes: float64(185), int64(2), object(4)
memory usage: 7.9+ GB


In [27]:
#!c1.8
all_cols = df.columns.to_list()
cat_cols = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']

dtype = {col: 'float16' for col in all_cols if col not in (cat_cols + ['customer_ID', 'S_2'])}

for col in cat_cols + ['customer_ID']:
    dtype[col] = 'category'
    


In [32]:
#!c1.8
for i in dtype.keys():
    if dtype[i]=='float16':
        df[i]=np.float16(df[i])

In [33]:
#!c1.8
gc.collect()

3

In [34]:
#!c1.8
joblib.dump(df,'small_df.sav')

['small_df.sav']

In [49]:
#!c1.8
CAT_COLUMNS = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']

NUMERIC_COLUMNS = [ i for i in dtype.keys() if dtype[i]=='float16']
NUMERIC_COLUMNS.remove('target')
catboost_features=CAT_COLUMNS+NUMERIC_COLUMNS

In [102]:
#!c1.8
train.fillna(-1)

,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,...,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145,target
650470,1e4a396a77c9f3bc595809716f0570cd4be39685734f48...,2017-08-31,0.643555,0.270996,0.011101,1.007812,0.008347,0.126465,0.122009,0.006149,...,-1.0,-1.0,0.009758,0.008209,0.008629,-1.000000,0.000362,0.003223,0.001394,0.0
3689326,aa724b2873dadc69e4c02d4d01bdc7573311c3d2caee6f...,2018-03-24,0.543945,0.355713,0.443359,0.025558,0.003366,0.216064,0.006668,0.739258,...,-1.0,-1.0,0.005718,0.004761,0.007889,-1.000000,0.000877,0.000265,0.000416,1.0
5011625,e7d4c5ef7004b988f21985a290a8eb97f02c4d7c492328...,2017-09-07,0.894531,0.416504,0.033051,1.000977,0.007790,0.150391,0.002085,0.006393,...,-1.0,-1.0,0.002041,0.007996,0.002890,-1.000000,0.005955,0.004543,0.008514,0.0
3241525,95bd90a0547f8f42a093a0ea442b920eca9bef9e514ce5...,2017-08-14,0.772461,0.003546,0.025269,0.813965,0.005962,0.057709,0.004494,0.011116,...,-1.0,-1.0,0.005867,0.007202,0.005505,-1.000000,0.007412,0.005386,0.004082,0.0
1552740,47858837194d4c527d2e78b7797fd50f0c534cad7911ef...,2017-05-14,0.672852,0.183716,0.041779,1.003906,0.001216,0.350342,0.009979,0.008041,...,-1.0,-1.0,0.003824,0.007252,0.004658,-1.000000,0.009514,0.001801,0.006798,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2261263,687c56c38126587e4ee0909b6e232657eabf89d07bd388...,2017-11-18,0.426514,0.002314,0.005577,0.815918,0.003101,-1.000000,0.003685,0.000048,...,-1.0,-1.0,1.009766,0.000230,0.867676,0.039795,1.004883,0.004978,0.098755,0.0
3688365,aa67270ff22d3c6ee4bab735d3fe9298534b6f0d3ad0a7...,2017-06-08,0.486084,0.008377,0.003860,0.810059,0.008347,-1.000000,0.000150,0.003393,...,-1.0,-1.0,0.007183,0.009125,0.008011,-1.000000,0.008774,0.008247,0.002357,0.0
1728331,4fc1f1d5b1ed9f35a5b1871a751daaa247778a9eccf562...,2017-10-07,0.223022,0.005001,0.110107,0.485352,1.003906,0.951660,0.009895,0.022903,...,-1.0,-1.0,0.007763,0.002855,0.004528,-1.000000,0.008896,0.004955,0.008965,1.0
1905530,57eb0de17baefbdf95e630d6f81358b2290335138e96aa...,2017-07-07,0.854004,0.271729,1.101562,1.000977,0.007046,0.075867,0.007343,0.001590,...,-1.0,-1.0,0.008957,0.002771,0.002815,-1.000000,0.000754,0.002644,0.005783,0.0


In [43]:
#!c1.8
train,val  = train_test_split(df,test_size=0.2,stratify=df['target'])

In [123]:
#!c1.8
for i in range(len(CAT_COLUMNS)):
    train[CAT_COLUMNS[i]]=[str(j)  for j in train[CAT_COLUMNS[i]]]
    val[CAT_COLUMNS[i]]=[str(j)  for j in val[CAT_COLUMNS[i]]]

In [232]:
#!c1.8
for i in range(len(CAT_COLUMNS)):
    test[CAT_COLUMNS[i]]=[str(j)  for j in test[CAT_COLUMNS[i]]]
    

In [106]:
#!c1.8
for i in [ ]:
    train[CAT_COLUMNS[i]]=[(np.int(i) if not np.isnan(i) else 'nan' ) for i in train[CAT_COLUMNS[i]]]
    val[CAT_COLUMNS[i]]=[(np.int(i) if not np.isnan(i) else 'nan' ) for i in val[CAT_COLUMNS[i]]]

TypeError: list indices must be integers or slices, not str

In [107]:
#!c1.8

train['target']=[(np.int(i) if not np.isnan(i) else 'nan' ) for i in train['target']]
val['target']=[(np.int(i) if not np.isnan(i) else 'nan' ) for i in val['target']]

In [105]:
#!c1.8
train['target']=np.int(train['target'])

TypeError: cannot convert the series to <class 'int'>

In [ ]:
#!c1.8
for i in CAT_COLUMNS:
    print(train[i].value_counts())

In [98]:
#!c1.8
model = CatBoostClassifier(random_seed=42,
                           iterations=10000,
                           learning_rate=0.02,
                           l2_leaf_reg=3,
                           depth=5,
                           #task_type="GPU", # раскомментить, когда с GPU
                           thread_count=-1,
                           use_best_model=True,
                           leaf_estimation_method='Newton',
                           custom_loss=[ 'AUC'],
                           nan_mode='Min'

                          )



In [124]:
#!c1.8
model.fit(train[catboost_features], train['target'],
                    cat_features = CAT_COLUMNS,
                    eval_set=(val[catboost_features], val['target']),
                    verbose=500, plot=False, early_stopping_rounds=600
         )

print(model.get_best_score())

0:	learn: 0.6726585	test: 0.6726691	best: 0.6726691 (0)	total: 3.39s	remaining: 9h 25m 3s
500:	learn: 0.2753480	test: 0.2754023	best: 0.2754023 (500)	total: 22m 21s	remaining: 7h 3m 47s
1000:	learn: 0.2695901	test: 0.2698207	best: 0.2698207 (1000)	total: 44m 29s	remaining: 6h 39m 59s
1500:	learn: 0.2668348	test: 0.2672275	best: 0.2672275 (1500)	total: 1h 6m 1s	remaining: 6h 13m 51s
2000:	learn: 0.2649395	test: 0.2654992	best: 0.2654992 (2000)	total: 1h 27m 32s	remaining: 5h 49m 58s
2500:	learn: 0.2634614	test: 0.2641926	best: 0.2641926 (2500)	total: 1h 49m 12s	remaining: 5h 27m 28s
3000:	learn: 0.2622335	test: 0.2631332	best: 0.2631332 (3000)	total: 2h 10m 39s	remaining: 5h 4m 43s
3500:	learn: 0.2611506	test: 0.2622200	best: 0.2622200 (3500)	total: 2h 32m 9s	remaining: 4h 42m 27s
4000:	learn: 0.2601602	test: 0.2613932	best: 0.2613932 (4000)	total: 2h 53m 39s	remaining: 4h 20m 22s
4500:	learn: 0.2592717	test: 0.2606634	best: 0.2606634 (4500)	total: 3h 14m 51s	remaining: 3h 58m 3s
5000:	

In [104]:
#!c1.8
train['target'].value_counts()

0.0    3322865
1.0    1102295
Name: target, dtype: int64

In [133]:
#!c1.8
train['t_est']=model.predict_proba(train[catboost_features])[:,1]

In [166]:
#!c1.8
1+1

2

In [235]:
#!c1.8
test['t_est']=model.predict_proba(test[catboost_features])[:,1]

In [129]:
#!c1.8
train['S_2']=pd.to_datetime(train['S_2'])
val['S_2']=pd.to_datetime(val['S_2'])

In [233]:
#!c1.8
1+1

2

In [234]:
#!c1.8
test['S_2']=pd.to_datetime(test['S_2'])

In [ ]:
#!c1.8
train.head()

In [ ]:
#!c1.8
for i in train['t_est'][:100]:
    print(i)

In [125]:
#!c1.8
###
Вернуть NA в числовые значения перед бустингом
Есть ли у нас пересечение теста и трейна
###

SyntaxError: invalid syntax (<ipython-input-1-77b1263849e5>, line 2)

In [ ]:
#!c1.8
1+1

In [163]:
#!c1.8
def preprocess(df,id,target=True):
    #print(df)
    df.sort_values('S_2',inplace=True)  # ascending = True по возрастанию
    estimates=df['t_est'].to_list()
    _list=[id,len(estimates)]
    
    _list=_list+[estimates[0]]*(18-len(estimates))+estimates
    if target:
        _list+=[df['target'].iloc[0]]
#     if len(_list)<20:
#         print('===')
    new_df=pd.DataFrame.from_records([_list])
    return new_df

In [ ]:
#!c1.8


    
regr_train=[]    
for id,df in train.groupby('customer_ID'):
    regr_train.append(preprocess(df,id))
    
regr_train=pd.concat(regr_train)
regr_train.columns=['customer_ID','len_list']+['col'+str(i) for i in range(1,19)]+['target']
    

In [150]:
#!c1.8
val['t_est']=model.predict_proba(val[catboost_features])[:,1]

In [151]:
#!c1.8
regr_train_val=[]    
for id,df in val.groupby('customer_ID'):
    regr_train_val.append(preprocess(df,id))
    
regr_train_val=pd.concat(regr_train_val)
regr_train_val.columns=['customer_ID','len_list']+['col'+str(i) for i in range(1,19)]+['target']

<ipython-input-1-54f578993ed8>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.sort_values('S_2',inplace=True)  # ascending = True по возрастанию


In [237]:
#!c1.8
regr_test=[]    
for id,df in test.groupby('customer_ID'):
    regr_test.append(preprocess(df,id,target=False))
    
regr_test=pd.concat(regr_test)
regr_test.columns=['customer_ID','len_list']+['col'+str(i) for i in range(1,19)]  #+['target']

<ipython-input-7-2b69cc5af793>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [148]:
#!c1.8
regr_train.columns=['customer_ID','len_list']+['col'+str(i) for i in range(1,19)]+['target']

In [146]:
#!c1.8
regr_train[110]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,00116df300d0b6762bf4661c53fa499bd4b8915b6a6636...,0.001616,0.006388,0.001061,0.002428,0.003391,0.00199,0.007352,0.004604,0.000655,0.000648,0.00049,0.000977,0


In [154]:
#!c1.8
train_clmns=regr_train.columns[1:-1]
regr_clf=LogisticRegression(C=5,max_iter=10000)
regr_clf.fit(regr_train[train_clmns],regr_train['target'])
probs=regr_clf.predict_proba(regr_train_val[train_clmns])[:,1]
print(roc_auc_score(regr_train_val['target'].values, probs))

0.9528831266308277


In [238]:
#!c1.8
1+1

2

In [239]:
#!c1.8
train_clmns=regr_train.columns[1:-1]
probs=regr_clf.predict_proba(regr_test[train_clmns])[:,1]
#print(roc_auc_score(regr_train_val['target'].values, probs))

In [169]:
#!c1.8
regr_test.head()

,customer_ID,len_list,col1,col2,col3,col4,col5,col6,col7,col8,col9,col10,col11,col12,col13,col14,col15,col16,col17,col18
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,9,0.118511,0.118511,0.118511,0.118511,0.118511,0.118511,0.118511,0.118511,0.118511,0.118511,0.123152,0.141278,0.155911,0.159883,0.117958,0.133614,0.096891,0.067206
0,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...,13,0.005840,0.005840,0.005840,0.005840,0.005840,0.005840,0.029305,0.006877,0.011171,0.005713,0.004922,0.004180,0.003705,0.004549,0.005037,0.004627,0.004572,0.004022
0,0000210045da4f81e5f122c6bde5c2a617d03eef67f82c...,13,0.087577,0.087577,0.087577,0.087577,0.087577,0.087577,0.089002,0.123580,0.116484,0.045691,0.040582,0.050107,0.037190,0.122938,0.216381,0.262812,0.307264,0.172739
0,00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976c...,13,0.716250,0.716250,0.716250,0.716250,0.716250,0.716250,0.705836,0.699904,0.664491,0.640767,0.553139,0.566970,0.669744,0.511646,0.490794,0.418733,0.510129,0.344996
0,00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...,13,0.691515,0.691515,0.691515,0.691515,0.691515,0.691515,0.689355,0.657174,0.699020,0.654079,0.653429,0.683605,0.821682,0.883017,0.799818,0.800576,0.843272,0.884379


In [240]:
#!c1.8
answer=regr_test[['customer_ID']].copy()
answer['prediction']=probs

In [241]:
#!c1.8
sub=pd.read_csv('sample_submission.csv')

In [242]:
#!c1.8
len(answer)

924621

In [236]:
#!c1.8
len(test.customer_ID.unique())  #924621

924621

In [243]:
#!c1.8
sub.drop('prediction',inplace=True,axis=1)

In [244]:
#!c1.8
sub=sub.merge(answer,left_on='customer_ID',right_on='customer_ID',how='left')

In [245]:
#!c1.8
sub.head()

,customer_ID,prediction
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,0.029033
1,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...,0.016619
2,0000210045da4f81e5f122c6bde5c2a617d03eef67f82c...,0.057743
3,00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976c...,0.234991
4,00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...,0.922792


In [246]:
#!c1.8
sub.to_csv('submission_17Aug-18-00.csv', index=None)

In [195]:
#!c1.8
len(sub)

93902

In [ ]:
#!c1.8
